In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
from tqdm.auto import tqdm
import plotly.express as px
import re

In [40]:
publishers = ["HarperCollins", "Little, Brown and Company", "Tor Books", "Doubleday", "Bloomsbury Publishing"]
# Disney Hyperion Books (Disney Publishing Group)、Oxford World's Classics 沒有書在榜
formats = ["Paperback", "Hardcover", "Kindle"]

In [46]:
data_all_books_with_price = pd.read_csv(r"amazon\data_book_price\Best_Sellers_top100_2dep_other_info.csv", encoding="latin-1")
data_all_books_with_category = pd.read_csv(r"amazon\data_book_price\BestSeller_2dep.csv", encoding="latin-1")

In [47]:
# 修改 data_all_books_with_price 的欄位名稱
data_all_books_with_price.rename(columns={"all_rank_list": "whole_rank", "before_discount": "original_price", "new_price": "selling_price"}, inplace=True)

# 修改 data_all_books_with_category 的欄位名稱
data_all_books_with_category.rename(columns={"department": "category", "rank": "category_rank"}, inplace=True)

# 修正 data_all_books_with_price 的資料格式
data_all_books_with_price["whole_rank"] = data_all_books_with_price["whole_rank"].apply(lambda x: int("".join(filter(str.isdigit, str(x)))) if "#" in str(x) else None)
data_all_books_with_price["format"] = data_all_books_with_price["format"].apply(lambda x: next((f for f in formats if f in x), "Other Format"))
data_all_books_with_price["original_price"] = data_all_books_with_price["original_price"].apply(
    lambda x: float("".join(filter(lambda y: str.isdigit(y) or y == ".", str(x)))) if "$" in str(x) else np.nan
)


def clean_selling_price(row):
    prices = re.findall(r"\$[0-9]+\.?[0-9]*", str(row["selling_price"]))
    if prices:
        return sum([float(str(price).replace("$", "")) for price in prices]) / len(prices)
    else:
        return row["original_price"]


data_all_books_with_price["selling_price"] = data_all_books_with_price.apply(clean_selling_price, axis=1)
data_all_books_with_price["publisher"] = data_all_books_with_price["other_list"].apply(lambda x: next((p for p in publishers if f"Publisher:{p}".lower() in str(x).lower()), "Other Publishers"))

# 修正 data_all_books_with_category 的資料格式
data_all_books_with_category["category_rank"] = data_all_books_with_category["category_rank"].apply(lambda x: int("".join(filter(str.isdigit, str(x)))) if "#" in str(x) else None)
data_all_books_with_category["format"] = data_all_books_with_category["format"].apply(lambda x: next((f for f in formats if f in x), "Other Format"))

In [48]:
data_all_books_with_price.head()

,whole_rank,format,original_price,selling_price,title,scrape_time,other_list,publisher
0,30397.0,Other Format,NaN,15.23,The Adventure Begins: The Early Classics (Adve...,08:05.6,Publisher:Tyndale Entertainment; Abridged ed...,Other Publishers
1,54341.0,Other Format,22.99,15.06,Spanish For Dummies Audio Set,08:15.5,Publisher:For Dummies; 1st edition (April 30...,Other Publishers
2,29496.0,Other Format,14.99,10.99,"28 Hours: 6 stories on Trust, Truth, and Disce...",08:24.2,"Publisher:Focus on the Family (October 4, 20...",Other Publishers
3,43543.0,Other Format,13.98,10.49,24K Magic,08:32.1,ASIN:B01LYRM74T Publisher:Atlantic (Januar...,Other Publishers
4,5.0,Kindle,NaN,14.99,The Women: A Novel,08:46.1,ASIN:B0C1X97LW7 Publisher:St. Martin's Pre...,Other Publishers


In [53]:
data_all_books_with_category.head()

,category,category_rank,title,book_url,author,format,price,scrape_time
0,Arts & Photography | Business of Art,1,The Lioness of Boston,https://www.amazon.com/Lioness-Boston-Emily-Fr...,Emily Franklin,Kindle,$10.99,31:07.1
1,Arts & Photography | Business of Art,2,2024 Vision Board Clip Art Book: Create Powerf...,https://www.amazon.com/2024-Vision-Board-Clip-...,Angelie Dane,Paperback,$17.04,31:07.1
2,Arts & Photography | Business of Art,3,Making It in the Art World: Strategies for Exh...,https://www.amazon.com/Making-Art-World-Approa...,Brainard Carey,Paperback,$19.99,31:07.1
3,Arts & Photography | Business of Art,4,How to Collect Art,https://www.amazon.com/How-Collect-Art-Magnus-...,Magnus Resch,Paperback,$24.49,31:07.1
4,Arts & Photography | Business of Art,5,Art/Work - Revised & Updated: Everything You N...,https://www.amazon.com/Art-Work-Revised-Update...,Heather Darcy Bhandari,Paperback,$14.39,31:07.1


In [54]:
data_all_books_whole_info = pd.merge(data_all_books_with_category, data_all_books_with_price, on=["title", "format"], how="left")

In [55]:
data_all_books_whole_info.to_excel("data_all_books_whole_info.xlsx", index=False)

In [ ]:
cloumns = ["category", "category_rank", "title", "format", "whole_rank", "original_price", "selling_price", "publisher"]
data_all_books_whole_info = data_all_books_whole_info[cloumns]
del data_all_books_with_category, data_all_books_with_price